# Projet de session - Analytique de la chaîne logistique

**Travail remis à :** Martin Cousineau

**Date de remise :** 8 décembre 2020

**Membres de l'équipe :** Alexis Caplette(11222588) - Kevin Hervieux(11275381) - Alexis Laplante(11220436) - Maylis Schwoerer(11203735)

# 1. Introduction : Présentation du contexte et du but du projet

**1.1 Mise en contexte**

Dans le cadre de ce projet de session pour le cours "Analytique de la chaîne logistique", notre équipe de quatre étudiants a été en mesure de mettre la main sur une base de données représentant les ventes effectuées par une chaîne de superstore se situant aux États-Unis. Pour des raisons de confidentialité, nous n'avons pas eu accès au nom exact de l'entreprise qui avait fourni ces données. Il est possible de retrouver des informations par rapport à quatre années de ventes différentes au sein de cette base de données, soit les années 2015, 2016, 2017 et 2018.

Selon les informations extraites dans la base de données, nous avons pu constater qu'il s'agissait d'une entreprise bien établie au niveau de la vente de produits de bureau. Parmi les sous-catégories présentes dans la base de données, on retrouve des accessoires, des appareils, de l'art, des cartables, des bibliothèques, des chaises, des photocopieuses, des enveloppes, des attaches, des meubles, des étiquettes, des machines, du papier, des téléphones, des outils de rangement, de l'équipement divers et des tables. Bref, on constate que ce superstore offre une gamme complète de produits afin de subvenir aux besoins de ses clients. Au niveau des clients, on constate que ceux-ci sont divisés sous trois catégories différentes, soit les entreprises, les particuliers et les bureaux à la maison. Comme nous le verrons plus tard dans ce rapport, les clients de l'entreprise se retrouvent en très grande majorité aux États-Unis à l'exception de quelques-uns se trouvant à l'international. Finalement, il nous a été possible de constater que l'entreprise offre quatre niveaux de livraison possible à ses clients, soit la journée même, la première classe, la deuxième classe ou la classe standard.

Ensuite, de manière plus précise, cette base de données à laquelle nous avions accès est divisée en 18 colonnes représentant le numéro d'identification de la commande, le numéro de la ligne, la date de commande, la date d'envoi, le type d'envoi, l'identifiant du client, le nom du client, le type de client, le pays, la ville, l'état, le code postal, la région, le numéro d'identification du produit, la catégorie du produit, la sous-catégorie du produit, le nom du produit et finalement, le montant de la vente. En ayant accès à toutes ces informations, nous étions confiants d'être en mesure de pouvoir tirer plusieurs analyses pertinentes de cette base de données. 

**1.2 But du projet**

Après avoir observé de manière globale la base de données à laquelle nous avions accès, nous avons déterminé qu'un but intéressant pour ce projet serait de créer un outil d'analyse descriptive permettant d'évaluer la situation des ventes de l'entreprise au cours des quatre années représentées dans la base de données. Par la suite, grâce à cet outil d'analyse, les dirigeants de l'entreprise pourraient être en mesure de mieux orienter leurs décisions stratégiques face à la localisation de leurs futurs centres de distribution. Pour ce faire, différents aspects sont évalués avec l'aide de cet outil d'analyse, soit les niveaux de commandes clients, les ventes et les expéditions en fonction des régions et le niveau de fidélité des clients au fil des années. Bien que cet outil d'analyse permettra aux dirigeants de l'entreprise d'effectuer eux-mêmes leurs propres constats, nous procéderons tout de même à plusieurs suggestions face aux résultats tirés de cet outil d'analyse afin d'en démontrer son utilité.



# 2. Méthodologie : Application et présentation de l'outil analytique

Dans la deuxième partie de ce rapport, nous allons vous présenter les différents codes utilisés afin de procéder à nos analyses. Chacune de ces cellules de code sera expliquée en détail dans le but d'expliquer le fonctionnement des différentes lignes de codes. L'analyse des différents résultats obtenus grâce aux nombreuses lignes de codes sera plutôt effectuée lors de la partie 3 de ce rapport.

**2.1 Initialisation du document**

Dans cette section, nous allons explorer les différentes étapes effectuées afin d'importer la base de données csv et les différents modules de python qui nous permettront d'élaborer nos analyses dans les sections suivantes.


La cellule de code ci-dessous permet d'importer les différents modules qui seront utilisés lors des nombreuses analyses effectuées dans le cadre de ce projet.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import seaborn as sns
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import dateutil.relativedelta
from geopy.geocoders import Nominatim
from pandas_profiling import ProfileReport
from geopy.extra.rate_limiter import RateLimiter
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
print("Setup Complete")

La cellule ci-dessous permet d'importer le fichier csv servant de base de données à ce projet. Il s'agit donc du fichier comprenant les données de ventes des quatre années mentionnées plutôt dans ce rapport. Après avoir importé le fichier csv, la cellule permet également d'afficher la base de données avec l'aide de Pandas en prenant la colonne du numéro d'identification des commandes comme index de référence.

In [ ]:
#Télécharger et lire la base de données.
my_filepath='../input/sales-forecasting/train.csv'
my_data = pd.read_csv(my_filepath,index_col='Order ID')
my_data

Finalement, cette dernière cellule de code pour la section d'initialisation du document permet de créer une copie de la base de données maîtresse afin d'être certain de ne pas modifier celle-ci de manière permanente en cas d'erreur. La base de données copiée se nommant my_data1 sera donc utilisée pour la suite afin d'effectuer les analyses. 

In [ ]:
#Mise en place d'une copie que nous allons utiliser pour coder.
my_data1 = my_data.copy()
print('Copy complete')

**2.2 Nettoyage de la base de données**

Pour cette section, nous allons procéder à plusieurs manipulations afin d'évaluer les données se trouvant dans le fichier auquel nous avons eu accès. De cette manière, nous allons pouvoir observer si les données étaient adéquates afin de procéder à nos analyses ou si des modifications devront être apportées à celles-ci.

Pour la cellule se trouvant ci-dessous, elle sert à identifier le type de données se trouvant dans chacune des colonnes de la base de données. Il est possible de constater que la majorité des données sont sous forme "object". Ce type de données pourrait causer un problème éventuellement lors de nos analyses effectuées à partir des colonnes "Order date" et "Ship date". Les données se trouvant dans ces colonnes devraient plutôt être présentées sous forme de "Datetime" afin que nous puissions extraire des délais par la suite. De plus, les données se trouvant dans la colonne "Postal code" pourraient porter problème éventuellement étant donné qu'elles sont représentées comme des "Float". Les codes postaux américains sont constitués d'une suite de cinq chiffres et ne devraient jamais être considérés comme des nombres à virgule.

In [ ]:
#Valider le type de données que nous avons.
print(my_data1.info())

La prochaine cellule sert à transformer le type des données se trouvant dans les colonnes "Order date" et "Ship date" afin qu'elles deviennent en type "Datetime" et non "Object". La première étape est d'importer le module datetime. Ensuite, la deuxième étape consiste à prendre les données se trouvant dans les colonnes "Order date" et "Ship date" et de les convertir en type "Datetime". Après avoir effectué ce changement, nous nous sommes rendu compte que les mois et les jours étaient inversés nous avons donc procédé à la correction des dates en inversant leurs valeurs lors de la troisième étape. La quatrième étape était de convertir à nouveau les dates en type "Datetime" étant donné que la correction effectuée lors de l'étape 3 avait remis les données en type "Object". Finalement la dernière étape était de sortir les informations par rapport aux données des différentes colonnes afin d'évaluer si les ajustements avaient été faits avec succès.

In [ ]:
#Importer le module datetime.
import datetime as dt

#Convertion de object en dates.
my_data1['Order Date'] =  pd.to_datetime(my_data1['Order Date'])
my_data1['Ship Date'] =  pd.to_datetime(my_data1['Ship Date'])


#La transformation cause certaines problématiques. En fait les jours et les mois furents inversés. Nous devons donc effectuer les lignes de codes suivantes.
#Inverser les jours et les mois dans notre cellule de date.
my_data1['Order Date'] = my_data1['Order Date'].apply(lambda x: dt.datetime.strftime(x, '%d-%m-%Y'))
my_data1['Ship Date'] = my_data1['Ship Date'].apply(lambda x: dt.datetime.strftime(x, '%d-%m-%Y'))

#Les lignes de codes présentées ci-dessus transformait le tout en object. Nous devons transformer à nouveaux les données en date.
#Re-conversion de object en dates.
my_data1['Order Date'] =  pd.to_datetime(my_data1['Order Date'])
my_data1['Ship Date'] =  pd.to_datetime(my_data1['Ship Date'])

#Valider que les colonnes sont bien converties.
print(my_data1.info())

Maintenant que les données des colonnes "Order date" et "Ship date" étaient transformées en dates nous pouvions ajouter une nouvelle colonne effectuant la différence entre "Ship date" et "Order date". Cette colonne nommée "Treatment time" nous sera très utile pour la suite de nos analyses étant donné qu'elle permet d'évaluer le temps de traitement en jours entre le moment où la commande est effectuée par le client et le moment où la commande est expédiée de l'entrepôt chez le client. Bref, la cellule ci-dessous permet dans un premier temps de soustraire le "Order date" au "Ship date" et retourne cette valeur dans la nouvelle colonne nommée "Treatment time". Dans un deuxième temps, le code présent dans la cellule retourne les premières lignes de la base de données afin de constater l'ajout de la nouvelle colonne.

In [ ]:
#Ajout de la colonne "Treatment time".
my_data1['Treatment time'] = my_data1['Ship Date']- my_data1['Order Date']
#Présentation de la base de données avec la colonne ajoutée.
my_data1.head()

Comme les données par rapport aux colonnes représentant des dates étaient corrigées, nous pouvions maintenant nous concentrer sur les données représentant les codes postaux. La cellule ci-dessous sert donc à représenter les différents codes postaux se trouvant dans la base de données et ce, de manière unique. Cette liste est également triée de façon croissante afin d'en faciliter l'observation. Grâce à cette liste, nous avons pu rapidement constater qu'il y avait plusieurs problèmes au niveau des données représentant les codes postaux. Premièrement, il est possible de constater que plusieurs codes postaux n'ont que quatre chiffres alors que les codes postaux américains sont tous composés de cinq chiffres. Deuxièmement, la liste comporte des codes postaux NAN qui représentent des codes postaux inexistants. Il s'agit donc de deux situations problématiques qui seront corrigées.


In [ ]:
#Exploration des codes postaux.
sorted(my_data1['Postal Code'].unique())

La cellule de code ci-dessous sert tout d'abord à supprimer les lignes de données comportant des codes postaux NAN. Comme nous n'avions pas accès à l'entreprise étant donné que la base de données ne dévoilait pas le nom de l'entreprise, nous ne pouvions procéder à la correction de ces codes postaux inexistants. En ayant accès à quelqu'un au sein de l'entreprise, il aurait fort probablement été possible de trouver les codes postaux associés à ces numéros de commandes afin de les corriger dans la base de données. Ensuite, la deuxième portion du code sert à transformer le type des données de "float" à "integer". Finalement, la dernière portion de code sert à valider que le type de données a bien été modifié en "integer".

In [ ]:
#Supprimer les données NAN.
my_data1 = my_data1[my_data1['Postal Code'].notna()]
my_data1['Postal Code'].isna().any()

#Transformer le code en integer.
my_data1['Postal Code']=my_data1['Postal Code'].astype(int)

#Valider que le code est bien integer.
print(my_data1.info())

La dernière modification à effectuer afin que notre base de données soit pleinement utilisable consistait à ajouter les 0 au début des codes postaux n'ayant seulement que quatre chiffres. La cellule de code ci-dessous permet donc dans un premier temps d'effectuer cet ajustement et dans un deuxième temps, elle permet encore une fois de représenter la liste des différents codes postaux se retrouvant dans la base de données après qu'ils aient été ajustés. Cette dernière étape nous permet de constater si nos corrections ont été effectuées avec succès.

In [ ]:
#Ajouter les 0 au début des codes postaux n'ayant pas 5 chiffres au total.
my_data1['Postal Code']=my_data1['Postal Code'].apply(lambda x: '{0:0>5}'.format(x))
sorted(my_data1['Postal Code'].unique())

**2.3 Analyse des commandes clients**

Dans cette section nous allons analyser le comportement des commandes clients afin de mieux pouvoir orienter nos suggestions par la suite. Il est utile pour l'équipe de gestion de bien saisir le comportement actuel de leurs clients, car cela permettra aux dirigeants de baser leurs décisions stratégiques qui affecteront le futur de l'entreprise en fonction de données représentatives et fiables par rapport à ce qui les attend pour la suite. Plus précisément, nous allons analyser le comportement des clients en fonction de plusieurs critères comme le type de clients et le type de produits par exemple.


Pour ce qui est de la cellule ci-dessous, elle permet dans un premier temps de sortir une analyse descriptive des ventes en prenant en compte plusieurs aspects comme la moyenne, le nombre de commandes effectuées, le minimum et le maximum par exemple. Cette évaluation globale nous permet d'avoir une idée générale du comportement des clients par rapport aux ventes effectuées. Dans un deuxième temps, cette cellule nous permet d'effectuer la même analyse, mais cette fois-ci en différenciant les trois segments différents déservis par l'entreprise. Cette analyse permet donc de constater les différents comportements auprès des trois types de clients retrouvés au sein de l'entreprise. 

In [ ]:
#Analyse descriptives du montant des ventes de manière générale. 
describe_analysis_sales = my_data1.Sales.describe()
print(describe_analysis_sales)

#Analyses descriptives du montant des ventes par segment. 
my_data1.groupby('Segment').Sales.describe()

Pour ce qui est de la cellule ci-dessous, elle permet dans un premier temps d'évaluer le nombre de commandes ayant été effectuées par chacun des segments. Ensuite, dans un deuxième temps elle permet de mettre ces informations sous forme de tableau. Finalement, le code permet d'ajouter une colonne indiquant le pourcentage des expéditions par rapport au total des expéditions qui est associé à chacune des catégories de client. Cette analyse pourra être très utile par la suite, car elle permettra aux dirigeants de constater quelle catégorie de client représente la plus grande proportion de ses expéditions.

In [ ]:
# Compter le nombre de commandes par segment.  
count_segment_type = my_data1.Segment.value_counts()
print(count_segment_type)

#Créer un tableau avec la formule de count_segment_type pour pouvoir le manipuler plus facilement.
segment_type = pd.DataFrame(count_segment_type)
segment_type.head()

# Créer une colonne "% of expedition" sur les commandes totales.
segment_type['% of expedition']=(segment_type['Segment']/segment_type.Segment.sum()*100)
segment_type.head()

Le code ci-dessous permet de générer le top 10 des meilleurs clients en fonctions du montant total des ventes effectuées auprès de ces clients. Dans un premier temps, le code permet d'évaluer le montant total des ventes pour chacun des clients. Dans un deuxième temps, un tableau représentant seulement les 10 clients ayant les ventes totales les plus élevées est créé. Cette analyse sera pertinente dans notre prise de décision stratégique étant donné que les dirigeants voudront fort probablement favoriser de bonnes relations d'affaires avec ces clients.

In [ ]:
#Regrouper le montant des ventes par client.
Top_client = pd.DataFrame(my_data1.groupby('Customer ID').Sales.sum())
print(Top_client)

#Afficher le top 10 des meilleurs clients.
Top10_sales = Top_client.nlargest(10, "Sales") 
Top10_sales

Le code ci-dessous permet de générer le top 10 des meilleurs clients en fonctions du nombre total d'expéditions effectuées auprès de ces clients. Dans un premier temps le code permet d'évaluer le nombre total d'expéditions pour chacun des clients. Dans un deuxième temps, un tableau représentant seulement les 10 clients ayant le nombre total d'expéditions le plus élevé est créé. Cette analyse sera pertinente dans notre prise de décision stratégique étant donné que les dirigeants pourront comparer s'ils veulent favoriser leurs clients en fonction du nombre d'expéditions ou plutôt en fonction de la valeur des ventes.

In [ ]:
#Regrouper les clients par nombre de commandes.
Nb_orders = pd.DataFrame(my_data1.value_counts('Customer ID'))
Nb_orders.columns = ['nb orders']
Nb_orders

#Afficher le top 10 des meilleurs clients.  
Top10_orders = Nb_orders.nlargest(10, "nb orders") 
Top10_orders

L'analyse ci-dessous permet de constater le type de livraison favorisé par les différents segments de clients qu'on retrouve au sein de l'entreprise. Le code permet donc d'additionner le nombre de commandes ayant été livrées selon chaque type de transport et ce, pour chaque segment de client. Cette analyse permettra aux dirigeants de concentrer leurs efforts au niveau de certains types de transport en fonction des clients majoritairement visés ou même de revoir la catégorisation des types de transport à l'avenir.

In [ ]:
#Représentation du type de livraison privilégié selon le segment.
client_delivery_type = my_data1.groupby(['Segment', 'Ship Mode']).Segment.agg([len])
pd.DataFrame(client_delivery_type)

La suite des trois cellules de code suivantes a été créée de la même manière pour chacune des cellules, cependant elles permettent de représenter des informations différentes. De manière générale, ces trois graphiques permettent de représenter sous forme de pourcentage la proportion des expéditions qui sont effectuées par chacun des types de transport pour un segment de client précis. Le premier graphique présente l'information pour le segment "Consumer", le deuxième graphique pour le segment "Corporate" et finalement le troisième graphique représente le segment "Home office". Dans un premier temps, le code permet de sélectionner seulement le segment de client désiré. Ensuite, dans un deuxième temps, une colonne représentant le pourcentage des expéditions selon le type de transport par rapport au total des expéditions de ce segment de client est ajoutée. Dans un troisième temps, le code permet de trier les pourcentages sous forme décroissante. Finalement, la dernière étape consiste à la création du graphique afin de représenter visuellement les résultats. Ces graphiques permettront donc aux dirigeants d'observer le comportement des différents segments de clients face aux types de livraisons qu'ils favorisent et permettront donc d'orienter leurs décisions pour le futur de l'entreprise encore une fois face aux segments de clients favorisés.

In [ ]:
#Afficher seulement les données du segment 'Consumer'.
ship_mode_conso = client_delivery_type.loc['Consumer']

#Créer une colonne "% of expedition" sur les commandes totales.
ship_mode_conso['% of expedition']=(ship_mode_conso['len']/ship_mode_conso.len.sum()*100)
print(ship_mode_conso.sort_values(by='len', ascending=False))


#Pour le graphique. 
#Trier les sous-produits par ordre décroissant.  
ship_mode_conso = ship_mode_conso.reset_index().sort_values(
    by='% of expedition', ascending=False).reset_index(drop=True)

#Génère la figure.
plt.figure(figsize=(14, 6))
graph = sns.barplot(data=ship_mode_conso, x='% of expedition', y='Ship Mode', orient='h', order=ship_mode_conso['Ship Mode'])

#Définit le titre.
plt.title("Nombre de commande par type de transport pour le segment 'Consumer'")

In [ ]:
#Afficher seulement les données du segment 'Corporate'.
ship_mode_corpo = client_delivery_type.loc['Corporate']

#Créer une colonne "% of expedition" sur les commandes totales.
ship_mode_corpo['% of expedition']=(ship_mode_corpo['len']/ship_mode_corpo.len.sum()*100)
print(ship_mode_corpo.sort_values(by='len', ascending=False))


#Pour le graphique.
#Trier les sous-produits par ordre décroissant.  
ship_mode_corpo = ship_mode_corpo.reset_index().sort_values(
    by='% of expedition', ascending=False).reset_index(drop=True)

#Génère la figure.
plt.figure(figsize=(14, 6))
graph = sns.barplot(data=ship_mode_corpo, x='% of expedition', y='Ship Mode', orient='h', order=ship_mode_corpo['Ship Mode'])

#Définit le titre.
plt.title("Nombre de commande par type de transport pour le segment 'Corporate'")

In [ ]:
#Afficher seulement les données du segment 'Home Office'.
ship_mode_home_office = client_delivery_type.loc['Home Office']

#Créer une colonne "% of expedition" sur les commandes totales.
ship_mode_home_office['% of expedition']=(ship_mode_home_office['len']/ship_mode_home_office.len.sum()*100)
print(ship_mode_home_office.sort_values(by='len', ascending=False))


#Pour le graphique. 
#Trier les sous-produits par ordre décroissant.  
ship_mode_home_office = ship_mode_home_office.reset_index().sort_values(
    by='% of expedition', ascending=False).reset_index(drop=True)

#Génère la figure.
plt.figure(figsize=(14, 6))
graph = sns.barplot(data=ship_mode_home_office, x='% of expedition', y='Ship Mode', orient='h', order=ship_mode_home_office['Ship Mode'])

#Définit le titre.
plt.title("Nombre de commande par type de transport pour le segment 'Home Office'")

La suite des trois cellules de code suivantes a été créée de la même manière pour chacune des cellules, cependant elles permettent de représenter des informations différentes. De manière générale, ces trois graphiques permettent de représenter sous forme de pourcentage la proportion des expéditions qui sont effectuées pour chacune des sous-catégories de produits et ce, pour un segment de client précis. Le premier graphique présente l'information pour le segment "Consumer", le deuxième graphique pour le segment "Corporate" et finalement le troisième graphique représente le segment "Home office". Dans un premier temps, le code permet de sélectionner seulement le segment de client désiré. Ensuite, dans un deuxième temps, une colonne représentant le pourcentage des expéditions selon la sous-catégorie de produit par rapport au total des expéditions de ce segment de client est ajoutée. Dans un troisième temps, le code permet de trier les pourcentages sous forme décroissante. Finalement, la dernière étape consiste à la création du graphique afin de représenter visuellement les résultats. Ces graphiques permettront donc aux dirigeants d'observer le comportement des différents segments de clients face aux sous-catégories de produits qu'ils favorisent et permettront donc d'orienter leurs décisions pour le futur de l'entreprise encore une fois face aux segments de clients favorisés.

In [ ]:
import matplotlib.pyplot as plt

produits_segments = (my_data1.groupby(['Segment','Sub-Category']).Segment.agg([len]))

#Afficher seulement les données des sous-catégories du segment 'Consumer'.
segment_conso = produits_segments.loc['Consumer']

#Créer une colonne "% of expedition" sur les commandes totales.
segment_conso['% of expedition']=(segment_conso['len']/segment_conso.len.sum()*100)
print(segment_conso.sort_values(by='len', ascending=False))

#Pour le graphique.
#Trier les sous-produits par ordre décroissant.  
segment_conso = segment_conso.reset_index().sort_values(
    by='len', ascending=False).reset_index(drop=True)

#Génère la figure.
plt.figure(figsize=(14, 6))
graph = sns.barplot(data=segment_conso, x='len', y='Sub-Category', orient='h', order=segment_conso['Sub-Category'])

#Définit le titre.
plt.title("Nombre de commande par sous-produits pour le segment 'Consumer'")

In [ ]:
#Afficher seulement les données des sous-catégories du segment 'Corporate'.
segment_corpo = produits_segments.loc['Corporate']
segment_corpo.sort_values(by='len', ascending=False)

#Créer une colonne "% of expedition" sur les commandes totales.
segment_corpo['% of expedition']=(segment_corpo['len']/segment_corpo.len.sum()*100)
print(segment_corpo.sort_values(by='len', ascending=False))


#Pour le graphique.
#Trier les sous-produits par ordre décroissant.  
segment_corpo = segment_corpo.reset_index().sort_values(
    by='len', ascending=False).reset_index(drop=True)

#Génère la figure.
plt.figure(figsize=(14, 6))
graph = sns.barplot(data=segment_corpo, x='len', y='Sub-Category', orient='h', order=segment_corpo['Sub-Category'])

#Définit le titre.
plt.title("Nombre de commande par sous-produits pour le segment 'Corporate'")

In [ ]:
#Afficher seulement les données des sous-catégories du segment  'Home Office'.
segment_home_office = produits_segments.loc['Home Office']

#Créer une colonne "% of expedition" sur les commandes totales.
segment_home_office['% of expedition']=(segment_home_office['len']/segment_home_office.len.sum()*100)
print(segment_home_office.sort_values(by='len', ascending=False))


#Pour le graphique.
#Trier les sous-produits par ordre décroissant.  
segment_home_office = segment_home_office.reset_index().sort_values(
    by='len', ascending=False).reset_index(drop=True)

#Génère la figure.
plt.figure(figsize=(14, 6))
graph = sns.barplot(data=segment_home_office, x='len', y='Sub-Category', orient='h', order=segment_home_office['Sub-Category'])

#Définit le titre.
plt.title("Nombre de commande par sous-produits pour le segment 'Home Office'")

**2.4 Ventes par région**

Maintenant que nous avons identifié le comportement des ventes en fonction des différents segments de clients, des types de livraison ou même des sous-catégories de produits, il était désormais temps d'évaluer le comportement des ventes en fonction des différentes régions desservies par l'entreprise.

La première cellule de code de cette section sert donc à représenter le montant total des ventes effectuées par état. Le code permet donc d'effectuer la somme des ventes pour chacun des états et affiche ensuite le tableau complet. Cette analyse nous sera utile afin de déterminer les états où se trouvent nos meilleurs clients au niveau des ventes.

In [ ]:
#Tableau de vente par état
Sales_per_state = pd.DataFrame(my_data1.groupby('State')['Sales'].sum())
Sales_per_state

La cellule ci-dessous ne nous permet pas de tirer d'analyses supplémentaires, mais elle sera utile lors des analyses suivantes. En fait, ce code permet d'associer la colonne "State" à l'index de ce tableau, ce qui nous sera utile par la suite afin de modifier le nom des états.

In [ ]:
#Ajout d'une colone représentant l'index.
Sales_per_state['State'] = Sales_per_state.index
Sales_per_state

Encore une fois, la cellule ci-dessous ne sert pas à effectuer des analyses, mais plutôt à ajuster nos données. Ce code permet de créer un dictionnaire assignant le nom complet des états à leur abréviation qui sera utilisée éventuellement dans nos analyses.

In [ ]:
#Création d'un dictionnaire pour assigner les abbréviations aux états. 
abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'}

La cellule ci-dessous permet également d'effectuer un changement au sein de notre dataframe afin de pouvoir utiliser ces données dans nos analyses par la suite. Ce code permet de prendre le dictionnaire créé dans la cellule précédente et d'associer l'abréviation à la colonne "State" selon l'état représenté dans l'index.

In [ ]:
#Modifier la colonne "State" pour l'abbréviation dans notre dataframe.
Sales_per_state['State'] = Sales_per_state.State.replace(abbrev)
Sales_per_state

La cellule de code ci-dessous permet de représenter sur une carte des États-Unis, l'intensité des ventes pour chacun des états avec l'aide d'une charte de couleurs. Plus la couleur est foncée vers le rouge, plus le volume des ventes est élevé dans cet état et plus la couleur est pâle vers le blanc, moins le volume des ventes est élevé dans cet état. Dans un premier temps, ce code permet d'importer les modules permettant de réaliser cette carte et d'utiliser pandas. Ensuite, dans un deuxième temps, le code permet d'établir les différents paramètres devant être pris en compte afin de produire la carte. Cette carte prend en compte le dataframe créé avec l'aide des cellules précédentes. Bien évidemment, cette carte permet aux gestionnaires de constater rapidement les états où l'entreprise effectue ses plus grands volumes de ventes, ce qui pourrait grandement influencer les futurs emplacements de ses centres de distribution. De plus, la carte est interactive, donc elle permet d'indiquer la somme totale des ventes pour chacun des états.

In [ ]:
import plotly.graph_objects as go

import pandas as pd

fig = go.Figure(data=go.Choropleth(
    locations=Sales_per_state['State'], #permet de sélectionner la colonne état abrégée de notre tableau
    z = Sales_per_state['Sales'].astype(float), 
    locationmode = 'USA-states', 
    colorscale = 'Reds',#La sélection de palette de couleur. Rouge = foncé
    colorbar_title = "Total Sales", #Nous colorions les états par les ventes totales
))

fig.update_layout(
    title_text = 'Ventes totales en dollars par états',
    geo_scope='usa', 
)

fig.show()

La cellule de code ci-dessous permet de dégager plusieurs statistiques auprès des ventes de chacun des états tels que le nombre de commandes effectuées, la moyenne du montant de ces commandes, le maximum et le minimum du montant des ces ventes. Ce code sert donc à regrouper les ventes par état et à extraire des statistiques dans un dataframe. Cette analyse pourra être utile pour les dirigeants si ces derniers souhaitent avoir accès à de l'information plus précise par rapport au comportement des ventes des différents états.

In [ ]:
#Analyse descriptive du montant des ventes par état.
my_data1.groupby('State').Sales.describe()

**2.5 Expédition des commandes selon les états**

Lors de la section précédente, nous avons analysé l'importance des différents états avec l'aide des niveaux de ventes. Toutefois,bien que nos ventes en dollars par état soient intéressantes à analyser, nous aimerions orienter nos décisions stratégiques plutôt sur l'aspect du nombre d'expéditions effectuées par état. Cette section sera donc consacrée à l'analyse de cet aspect plutôt axé sur la logistique et non sur les ventes et marketing.

Le code ci-dessous permet d'évaluer le nombre de commandes effectuées auprès des cinq villes les plus populaires dans la base de données. Le code calcule dans un premier temps le nombre total de commandes effectuées pour chacune des villes présentes dans la base de données et retourne ensuite un tableau démontrant le nombre de commandes effectuées auprès des cinq villes les plus populaires de la base de données. D'un point de vue logistique, ces informations vont être très importantes pour les dirigeants étant donné qu'elles permettent de constater les cinq points de vente principaux où les commandes de l'entreprise doivent être expédiées. Il pourrait donc être intéressant de considérer placer leurs centres de distribution à proximité de ces villes. 

In [ ]:
#Voir le top 5 des villes les plus demandées.

#Count du nombre de transports effectués des USA vers une déstination précice.
count_destination_country=my_data1.City.value_counts()
city_destination = pd.DataFrame(count_destination_country)
city_destination.head()

La suite des cinq cellules de code suivantes a été créée de la même manière pour chacune des cellules, cependant elles permettent de représenter des informations différentes. De manière générale, ces cinq tableaux permettent de représenter sous forme de pourcentage la proportion des expéditions qui sont effectuées pour chacune des sous-catégories de produit et ce, pour une ville précise. Le premier tableau présente l'information pour la ville de "New York City", le deuxième tableau présente l'information pour la ville de "Los Angeles", le troisième tableau présente l'information pour la ville de "Philadelphia", le quatrième tableau présente l'information pour la ville de "San Francisco" et finalement le cinquième tableau présente l'information pour la ville de "Seattle". Tout d'abord, le code permet de représenter le nombre d'expéditions par sous-catégorie de produits. Ensuit, le code permet de sélectionner seulement la ville désirée. Le code suivant ajoute une colonne représentant le pourcentage des expéditions selon la sous-catégorie de produits par rapport au total des expéditions de cette ville. Finalement, le dernier code permet de trier les pourcentages sous forme décroissante. Ces tableaux permettront donc aux dirigeants d'observer le comportement des cinq principales villes desservies par l'entreprise face aux sous-catégories de produits qui sont favorisées par les clients de ces villes.

In [ ]:
#Présenter les nombre de commandes par sous-categorie pour 'NYC'.
#Sous-Catégories de produits par ville.
sub_category_city = (my_data1.groupby(['City','Sub-Category']).Segment.agg([len]))

#Afficher seulement les données pour la ville de New York City.
sub_category_NYC = sub_category_city.loc['New York City']

#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_NYC['% of expedition']=(sub_category_NYC['len']/sub_category_NYC.len.sum()*100)
pd.DataFrame(sub_category_NYC).sort_values(by='len', ascending=False)

In [ ]:
#Afficher seulement les données pour la ville de Los Angeles. 
sub_category_LA = sub_category_city.loc['Los Angeles']


#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_LA['% of expedition']=(sub_category_LA['len']/sub_category_LA.len.sum()*100)
pd.DataFrame(sub_category_LA).sort_values(by='len', ascending=False)

In [ ]:
#Afficher seulement les données pour la ville de Philadelphia.
sub_category_PHL = sub_category_city.loc['Philadelphia']


#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_PHL['% of expedition']=(sub_category_PHL['len']/sub_category_PHL.len.sum()*100)
pd.DataFrame(sub_category_PHL).sort_values(by='len', ascending=False)

In [ ]:
#Afficher seulement les données pour la ville de San Francisco.
sub_category_SF = sub_category_city.loc['San Francisco']


#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_SF['% of expedition']=(sub_category_SF['len']/sub_category_SF.len.sum()*100)
pd.DataFrame(sub_category_SF).sort_values(by='len', ascending=False)

In [ ]:
#Afficher seulement les données pour la ville de Seattle.
sub_category_SEA = sub_category_city.loc['Seattle']


#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_SEA['% of expedition']=(sub_category_SEA['len']/sub_category_SEA.len.sum()*100)
pd.DataFrame(sub_category_SEA).sort_values(by='len', ascending=False)

La suite des quatre cellules de code suivantes a été créée de la même manière pour chacune des cellules, cependant elles permettent de représenter des informations différentes. De manière générale, ces quatre tableaux permettent de représenter sous forme de pourcentage la proportion des expéditions qui sont effectuées pour chacune des catégories de produits dans un premier temps et pour chacune des sous-catégories de produits dans un deuxième temps et ce, pour une région précise. Le premier tableau présente l'information pour la région "Central", le deuxième tableau présente l'information pour la région "East", le troisième tableau présente l'information pour la région "South" et finalement le quatrième tableau présente l'information pour la région "West". Tout d'abord, le code permet de sélectionner seulement le segment de client désiré. Ensuite, une colonne représentant le pourcentage des expéditions selon la catégorie de produit par rapport au total des expéditions de cette région est ajoutée. Finalement, le code suivant permet de trier les pourcentages sous forme décroissante. Ces trois étapes sont par la suite répétées pour les sous-catégories de produits de cette même région. Ces tableaux permettront donc aux dirigeants d'observer le comportement des différentes régions face aux catégories et mêmes sous-catégories de produits qu'ils favorisent en fonction des différentes régions visées par les centres de distribution.

In [ ]:
#Avoir plus de détails pour les catégories de produit pour la région 'Central'. 
#Afficher seulement les données pour la region 'Central'.

#Catégories de produits par région.
products_region = (my_data1.groupby(['Region','Category']).Segment.agg([len]))
print(products_region)

#Sous-Catégories de produits par région.
sub_category_region = (my_data1.groupby(['Region','Sub-Category']).Segment.agg([len]))
category_central = products_region.loc['Central']

#Créer une colonne "% of expedition" sur les commandes totales.
category_central['% of expedition']=(category_central['len']/category_central.len.sum()*100)
print(category_central.sort_values(by='len', ascending=False))


#Avoir plus de détails pour les sous- catégories de produits pour la région 'Central'. 
#Afficher que les données pour la region 'Central'
sub_category_central = sub_category_region.loc['Central']

#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_central['% of expedition']=(sub_category_central['len']/sub_category_central.len.sum()*100)
print(sub_category_central.sort_values(by='len', ascending=False))


In [ ]:
#Avoir plus de détails pour les catégories de produits la région 'East'.
#Afficher seulement les données pour la region 'East'.
category_east = products_region.loc['East']

#Créer une colonne "% of expedition" sur les commandes totales.
category_east['% of expedition']=(category_east['len']/category_east.len.sum()*100)
print(category_east.sort_values(by='len', ascending=False))

#Avoir plus de détails pour les sous-catégories de produits pour la région 'East'.
sub_category_east = sub_category_region.loc['East']

#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_east['% of expedition']=(sub_category_east['len']/sub_category_east.len.sum()*100)
print(sub_category_east.sort_values(by='len', ascending=False))


In [ ]:
#Avoir plus de détails pour la région 'South'.
#Afficher seulement les données pour la region 'South'.
category_south = products_region.loc['South']

#Créer une colonne "% of expedition" sur les commandes totales.
category_south['% of expedition']=(category_south['len']/category_south.len.sum()*100)
print(category_south.sort_values(by='len', ascending=False))


#Avoir plus de détails pour les sous-catégories de produits pour la région 'South'.

#Afficher seulement les données pour la region 'South'.
sub_category_south = sub_category_region.loc['South']

#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_south['% of expedition']=(sub_category_south['len']/sub_category_south.len.sum()*100)
print(sub_category_south.sort_values(by='len', ascending=False))


In [ ]:
#Afficher seulement les données pour la region 'West'.
category_west = products_region.loc['West']

#Créer une colonne "% of expedition" sur les commandes totales.
category_west['% of expedition']=(category_west['len']/category_west.len.sum()*100)
print(category_west.sort_values(by='len', ascending=False))


#Avoir plus de détails pour les sous-catégories de produits pour la région 'West'.
#Afficher seulement les données pour la region 'West'.
sub_category_west = sub_category_region.loc['West']

#Créer une colonne "% of expedition" sur les commandes totales.
sub_category_west['% of expedition']=(sub_category_west['len']/sub_category_west.len.sum()*100)
print(sub_category_west.sort_values(by='len', ascending=False))

La cellule de code suivante de cette section sert donc à représenter le nombre total d'expéditions effectuées par état. Le code permet donc d'effectuer la somme des expéditions pour chacun des états et affiche ensuite le tableau complet. Cette analyse nous sera utile afin de déterminer les états où se trouvent nos meilleurs clients au niveau des expéditions.

In [ ]:
#Tableau d'expéditions par état.
Delivery_per_state = pd.DataFrame(my_data1.groupby('State')['Sales'].count())
Delivery_per_state.rename(columns={"Sales": "Delivery"})
Delivery_per_state

La cellule ci-dessous ne nous permet pas de tirer d'analyses supplémentaires, mais elle sera utile lors des analyses suivantes. En fait, ce code permet d'associer la colonne "State" à l'index de ce tableau, ce qui nous sera utile par la suite afin de modifier le nom des états.

In [ ]:
#Ajout d'une colone représentant l'index.
Delivery_per_state['State'] = Delivery_per_state.index
Delivery_per_state

La cellule ci-dessous permet d'effectuer un changement au sein de notre dataframe afin de pouvoir utiliser ces données dans nos analyses par la suite. Ce code permet de prendre le dictionnaire créé précédemment dans cette analyse et d'associer l'abréviation à la colonne "State" selon l'état représenté dans l'index.

In [ ]:
#Modifier la colonne 'State' pour l'abbréviation dans notre dataframe.
Delivery_per_state['State'] = Delivery_per_state.State.replace(abbrev)
Delivery_per_state

La cellule de code ci-dessous permet de représenter sur une carte des États-Unis, l'intensité des expéditions pour chacun des états avec l'aide d'une charte de couleurs. Plus la couleur est foncée vers le rouge, plus le volume des expéditions est élevé dans cet état et plus la couleur est pâle vers le blanc, moins le volume des expéditions est élevé dans cet état. Dans un premier temps, ce code permet d'importer les modules permettant de réaliser cette carte et d'utiliser pandas. Ensuite, dans un deuxième temps, le code permet d'établir les différents paramètres devant être pris en compte afin de produire la carte. Cette carte prend en compte le dataframe créé avec l'aide des cellules précédentes. Bien évidemment, cette carte permet aux gestionnaires de constater rapidement les états où l'entreprise effectue ses plus grands volumes d'expéditions, ce qui pourrait grandement influencer les futurs emplacements de ses centres de distribution étant donné que cela signifie que de nombreux coûts de transports sont générés afin de subvenir aux demandes des clients se trouvant dans ces états. De plus, la carte est interactive, donc elle permet d'indiquer la somme totale des expéditions pour chacun des états.

In [ ]:
import plotly.graph_objects as go

import pandas as pd

fig = go.Figure(data=go.Choropleth(
    locations=Delivery_per_state['State'], #permet de sélectionner la colonne état abrégée
    z =Delivery_per_state['Sales'].astype(float), 
    locationmode = 'USA-states', 
    colorscale = 'Reds',#La sélection de palette de couleur. Plus il a de vente, plus c'est foncé
    colorbar_title = "Total Delivery", #Nous colorions les états par les ventes totales
))

fig.update_layout(
    title_text = 'Expéditions par état',
    geo_scope='usa', 
)

fig.show()

Les trois prochaines cellules de codes ci-dessous permettront d'effectuer les mêmes analyses. La seule différence se retrouvera au niveau de l'état étant donné que la même analyse sera effectuée auprès des trois états mentionnés ci-dessus. Ces analyses permettent de démontrer les 10 villes où l'entreprise effectue le plus grand volume d'expéditions au sein des états de la Californie, New York et le Texas. Dans un premier temps, le code permet de regrouper le nombre de fois où une expédition est effectuée vers une ville en particulier. Ensuite, le code effectue un classement en ordre décroissant des villes selon le nombre d'expéditions. Dans un troisième temps, le code permet de sélectionner seulement les données pour lesquelles l'état de référence est la Californie. Finalement, le code permet de représenter graphiquement seulement les 10 villes où le volume d'expéditions est le plus élevé. Cette analyse sera très pertinente afin de nous aider à préciser les recommandations que nous effectuerons aux dirigeants. Effectivement, plus tôt dans ce rapport, nous avions ciblé les états les plus populaires, mais ces analyses supplémentaires nous permettent maintenant de cibler directement une ville au sein de ces états afin de possiblement y installer un centre de distribution.

In [ ]:
#Présenter les villes déservies le plus souvent par l'état de la Californie.
top_state = my_data1.groupby(['State', 'City']).State.agg([len])
pd.DataFrame(top_state).sort_values(by='len',ascending=False)
top1 = top_state.loc['California'].sort_values(by='len',ascending=False)
top1.head(10)

In [ ]:
# Présenter les villes déservies le plus souvent pour l'état de New-York
top_state = my_data1.groupby(['State', 'City']).State.agg([len])
pd.DataFrame(top_state).sort_values(by='len',ascending=False)
top2 = top_state.loc['New York'].sort_values(by='len',ascending=False)
top2.head(10)

In [ ]:
# Présenter les villes déservies le plus souvent pour l'état de Texas
top_state = my_data1.groupby(['State', 'City']).State.agg([len])
pd.DataFrame(top_state).sort_values(by='len',ascending=False)
top3 = top_state.loc['Texas'].sort_values(by='len',ascending=False)
top3.head(10)

**2.6 Analyse des destinations desservies**

Maintenant que nous avons été en mesure de déterminer les états où le volume des expéditions était le plus élevé soit la Californie, le Texas et New York, il nous semblait intéressant d'aller raffiner nos recherches au sein de ces états. Pour ce faire, nous avons décidé d'aller observer le comportement des principales villes se trouvant dans ces états. 

La cellule de code ci-dessous ne sert pas à effectuer directement des analyses, mais elle permet d'effectuer des manipulations qui seront utiles pour la suite de nos analyses. Dans un premier temps, le code permet de créer une variable "adresse" afin que celle-ci soit utilisée dans Nominatim par la suite. Dans un deuxième temps, Nominatim permet de trouver la longitude, la latitude et l'altitude des différentes adresses clients afin de pouvoir les repérer rapidement sur une carte. Ensuite, le code permet de regrouper ces informations dans un dataframe ayant une colonne pour la longitude, la latitude et l'altitude. Une colonne 'Point' est également créée afin de de regrouper les trois composantes de l'emplacement exact du client en une seule valeur. Finalement, comme nous utilisons une version gratuite de Nominatim, nous devons mettre un délai minimum d'une seconde par transaction. Pour éviter que ce délai soit présent à chaque fois que la variable 'adresse' est utilisée, nous avons créé la variable 'adresse_backup' qui rapporte le dataframe 'adresse' qui a été créé avec Nominatim.

In [ ]:
adresse=my_data1
adresse['complete adresse']=my_data1['State']+','+my_data1['City']+','+my_data1['Postal Code']
adresse=pd.DataFrame(data=adresse).rename(columns={0:'complete adresse'}).reset_index().dropna().drop('Order ID',axis=1)
adresse=adresse.groupby('complete adresse').Sales.sum()
adresse=pd.DataFrame(data=adresse).reset_index()
adresse
locator = Nominatim(user_agent='kevhervieux@gmail.com')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)#on pourrait etre beaucoup plus rapide mais on doit payer donc delai min = 1 sec
adresse['location'] = adresse['complete adresse'].apply(geocode)
adresse['point'] = adresse['location'].apply(lambda loc: tuple(loc.point) if loc else None)
adresse[['latitude', 'longitude', 'altitude']] = pd.DataFrame(adresse['point'].tolist(), index=adresse.index)
adresse_backup=adresse

Encore une fois, cette cellule de code ne permet pas d'effectuer des analyses, mais plutôt à préparer les données pour nos analyses à venir. Cette fois-ci le code permet dans un premier temps de sélectionner uniquement les données pour lesquelles nous avons la longitude et la latitude de l'emplacement client. Les clients pour lesquels nous n'avons pas réussi à générer l'une ou l'autre de ces valeurs ne seront donc pas pris en compte étant donné que ces valeurs sont essentielles afin que nous puissions représenter les clients sur une carte avec l'outil Folium. Ensuite, le code permet de réinitialiser la colonne 'Index' maintenant que certaines adresses incomplètes ont été enlevées et il permet également d'afficher le dataframe corrigé. 

In [ ]:
adresse=adresse_backup
adresse=adresse[adresse['longitude'].notnull()]
adresse=adresse[adresse['latitude'].notnull()]
adresse=adresse.reset_index()
adresse

Le code ci-dessous permet de localiser les différents clients de l'entreprise sur une carte des États-Unis. Avec l'aide de la fonction Folium, le code sélectionne la latitude et la longitude de chacun des clients présents dans le dataframe 'adresse' et il indique sur une carte du monde avec l'aide d'un marqueur l'emplacement exact de ces clients. Ensuite, le code permet d'afficher la carte qui vient d'être créée. Cette analyse pourra être utilisée par les dirigeants de l'entreprise afin de vérifier l'entendu du territoire couvert par les activités de l'entreprise.

In [ ]:
us_map=folium.Map(location=[37.0902, -95.7129], zoom_start=4,tiles='cartodbpositron')
mc=MarkerCluster()
client=folium.map.FeatureGroup()
for lat, lng, in zip(adresse['latitude'], adresse['longitude']):
    client.add_child(
        folium.features.Marker([lat, lng]))
us_map.add_child(client)
us_map

Comme la carte générée ci-dessus était plutôt difficile à interpréter lorsque plusieurs points de clients se retrouvaient au même endroit, nous avons créé la "Heat map" ci-dessous afin de mieux représenter la situation. Encore une fois avec la fonction Folium, le code permet de prendre en compte la latitude et la longitude de chacun des clients présents dans le dataframe 'adresse' et il indique leur emplacement sur une carte du monde en utilisant des zones cette fois-ci. Le code nous permet de sélectionner le rayon désiré dans lequel les différents clients sont regroupés. Après avoir essayé des rayons de 5-10-15-20 et 25, nous avons déterminé que le rayon le plus adéquat était celui de 15. Si la couleur d'une région est chaude donc se rapprochant du rouge, cela signifie qu'il y a une forte densité de clients dans cette région. À l'inverse, si la couleur est plutôt froide donc se rapprochant du vert, cela signifie qu'il y a une faible densité de clients dans cette région. Finalement, le code permet d'afficher la carte qui vient d'être créée. Cette analyse pourra donc être très utile afin d'évaluer la densité des marchés desservis par l'entreprise. De cette manière, les dirigeants pourront orienter leurs décisions dans le but de viser des marchés de clients précis où la densité est plus élevée. En visant ce type de marchés denses, cela permet à une entreprise de limiter ses déplacements et de ce fait ses coûts de transport tout en ayant accès au plus grand volume de clients possible.

In [ ]:
us_heat_map=folium.Map(location=[37.0902, -95.7129], zoom_start=4, tiles='cartodbpositron')
HeatMap(data=adresse[['latitude','longitude']],radius=15).add_to(us_heat_map)#Tentatives avec 5-10-15-20-25 et 15 donne le meilleur résultat.
us_heat_map

**2.7 Analyse des clients perdus**

Finalement, la dernière section de notre analyse sera consacrée à l'évaluation des clients qui ont cessé de commander auprès de l'entreprise au fil des années. Comme nos analyses sont grandement orientées vers la prise de décisions par rapport aux emplacements potentiels des centres de distribution, il nous semblait intéressant d'évaluer le nombre de clients ayant cessé de commander auprès de l'entreprise afin de voir où se trouvaient géographiquement ces clients. 

La cellule de code ci-dessous permet de créer une liste regroupant tous les clients n'ayant pas effectué une commande au court des x nombre de mois et d'années désirés. Le code a été créé afin de permettre à l'utilisateur de saisir manuellement le nombre de mois et d'années auxquels il veut reculer dans le temps afin de constater les clients qui n'ont pas effectué de commandes auprès de l'entreprise dans ce laps de temps. La date de référence afin de reculer dans le temps est donc la date la plus récente de la base de données. Dans un premier temps, le code établit que la variable 'date_today' équivaut à la date la plus récente de la base de données. Ensuite, le code permet d'évaluer en fonction des paramètres saisis manuellement dans la fonction (nombre x de mois et y d'années dont l'utilisateur veut reculer dans le temps) quels identifiants clients se retrouvant dans la base de données complète n'ont pas effectué de commandes lors de cette période de x mois et y années. Dans un troisième temps, le code associe les clients n'ayant pas effectué de commandes au cours de cette période de x mois et y années à un dataframe. Finalement, le dataframe est représenté sous forme de tableau à une colonne. Les paramètres x et y représentant les mois et les années ont été configurés de cette manière afin que les dirigeants puissent évaluer plusieurs scénarios à leur guise. Cette analyse peut être utile afin d'avoir la liste des clients n'ayant pas effectué de commandes au cours d'une certaine période de temps. En ayant cette information entre leurs mains, les dirigeants de l'entreprise peuvent évaluer s'il s'agit de clients majeurs et si des démarches doivent être faites afin de regagner la confiance de ces clients. Cette analyse sera également très utile par la suite afin de représenter graphiquement ces clients perdus.

In [ ]:
date_today=my_data1['Order Date'].max()


def client_perdu(data_frame,nom_colonne,mois_derniere_commande,annee_derniere_commande,colonne_client):#t_derniere_commande est en mois.
    f = dateutil.relativedelta.relativedelta(months=mois_derniere_commande)
    g = dateutil.relativedelta.relativedelta(years=annee_derniere_commande)
    date_max=date_today-f-g
    date_max=np.datetime64(date_max)
    x= data_frame.loc[data_frame[nom_colonne]>=date_max]
    x=x[colonne_client]
    x=pd.DataFrame(data=x).drop_duplicates(subset=colonne_client).set_index(colonne_client)
    total_client=data_frame[colonne_client]
    total_client=pd.DataFrame(total_client).drop_duplicates(subset=colonne_client).set_index(colonne_client)
    x=total_client[~total_client.index.isin(x.index)]
    return x
client_perdu(my_data1,'Order Date',0,1,'Customer ID')

La cellule de code suivante permet quant à elle de prendre la liste des clients perdus créée dans la cellule précédente et de ressortir les différentes commandes ayant été effectuées historiquement par ces clients. Dans un premier temps, le code permet d'établir l'index de la base de données comme étant l'identifiant client. Ensuite, le code fait en sorte de comparer les identifiants clients se retrouvant dans le dataframe des clients perdus avec les identifiants clients du dataframe de la base de données complète afin de ne sélectionner seulement ceux qui se retrouvent dans les deux dataframes. Les identifiants clients sélectionnés sont ensuite regroupés dans un nouveau dataframe où il est possible de constater les différentes commandes ayant été effectuées par le passé par ces clients perdus. Cette analyse est donc pertinente afin d'évaluer le comportement historique de ces clients n'ayant pas commandé auprès de l'entreprise depuis un certain temps. De cette manière, il est donc plus facile d'évaluer s'il s'agit de clients habitués qui auraient vécu une expérience peu agréable ou s'il s'agit seulement de clients peu habituels. 

In [ ]:
client_perdu_clean=client_perdu(my_data1,'Order Date',0,1,'Customer ID')
total_client=my_data1.set_index('Customer ID')
client_perdu_clean=total_client[total_client.index.isin(client_perdu_clean.index)]#868 commandes que ces clients on passer, il y a un probleme?
client_perdu_clean=client_perdu_clean.merge(right=adresse,how='left',on='complete adresse')
client_perdu_clean.dropna()

Le code ci-dessous permet de localiser les différents clients de l'entreprise qui se retrouvent dans le dataframe créé ci-dessus sur une carte des États-Unis. Il s'agit donc des clients n'ayant pas effectué de commandes auprès de l'entreprise depuis un certain temps. Avec l'aide de la fonction Folium, le code sélectionne la latitude et la longitude de chacun des clients présents dans le dataframe ayant été créé dans la cellule ci-dessus et il indique sur une carte du monde avec l'aide d'un marqueur l'emplacement exact de ces clients. Ensuite, le code permet d'afficher la carte qui vient d'être créée. Cette analyse pourra être utilisée par les dirigeants de l'entreprise afin de vérifier l'emplacement des clients n'ayant possiblement pas été satisfaits de leur dernière expérience avec l'entreprise.

In [ ]:
client_perdu_clean=client_perdu_clean[client_perdu_clean['longitude'].notnull()]
client_perdu_clean=client_perdu_clean[client_perdu_clean['latitude'].notnull()]
client_perdu_clean_map=folium.Map(location=[37.0902, -95.7129], zoom_start=4, tiles='cartodbpositron')
mc=MarkerCluster()
client=folium.map.FeatureGroup()
for lat, lng, in zip(client_perdu_clean['latitude'], client_perdu_clean['longitude']):
    client.add_child(
        folium.features.Marker([lat, lng]))
client_perdu_clean_map.add_child(client)
client_perdu_clean_map

Comme la carte générée ci-dessus était plutôt difficile à interpréter lorsque plusieurs points de clients se retrouvaient au même endroit, nous avons créé la "Heat map" ci-dessous afin de mieux représenter la situation des clients perdus. Encore une fois avec la fonction Folium, le code permet de prendre en compte la latitude et la longitude de chacun des clients présents dans le dataframe des clients perdus et il indique leur emplacement sur une carte du monde en utilisant des zones cette fois-ci. Le code nous permet de sélectionner le rayon désiré dans lequel les différents clients sont regroupés. Après avoir essayé des rayons de 5-10-15-20 et 25, nous avons déterminé que le rayon le plus adéquat était celui de 15. Si la couleur d'une région est chaude donc se rapprochant du rouge, cela signifie qu'il y a une forte densité de clients dans cette région. À l'inverse, si la couleur est plutôt froide donc se rapprochant du vert, cela signifie qu'il y a une faible densité de clients dans cette région. Finalement, le code permet d'afficher la carte qui vient d'être créée. Cette analyse pourra donc être très utile afin d'évaluer la densité des marchés où les clients semblent possiblement être le moins satisfaits. De cette manière, les dirigeants pourront orienter leurs décisions dans le but de viser des marchés de clients précis pour lesquels ils veulent augmenter les efforts afin de conserver la fidélité de ces clients.

In [ ]:
client_perdu_clean_heat_map=folium.Map(location=[37.0902, -95.7129], zoom_start=4, tiles='cartodbpositron')
HeatMap(data=client_perdu_clean[['latitude','longitude']],radius=15).add_to(client_perdu_clean_heat_map)#j<ai essaye 5-10-15-20-25 et 15 c<est le plus sexy
client_perdu_clean_heat_map

L'analyse effectuée dans la cellule ci-dessous permet de déterminer le nombre d'expéditions par type de livraison pour lesquelles les clients perdus n'ont possiblement pas été satisfaits. Le code permet donc de créer une liste indiquant le nombre d'expéditions regroupées par le type de transport à partir du dataframe des expéditions historiques des clients n'ayant pas commandé auprès de l'entreprise depuis un certain temps. Cette analyse est pertinente afin de déterminer le type de transport qui a été utilisé par les clients ayant possiblement été insatisfaits. Les dirigeants peuvent ensuite évaluer si ce type de transport répond aux attentes des clients ou si ceux-ci sont mal informés par rapport à la performance de ce type de livraison.

In [ ]:
analyse_cp=client_perdu_clean
analyse_cp['Ship Mode'].value_counts()

# 3. Analyse des résultats obtenus

La troisième partie de ce rapport permettra d'explorer les résultats obtenus avec l'aide des analyses effectuées lors de la partie précédente de ce rapport. Nous allons donc expliquer les résultats obtenus afin de démontrer le type de décisions qui pourraient découler de nos analyses. Ces explications seront donc présentées sous forme de recommandations stratégiques aux dirigeants de l'entreprise afin de démontrer l'utilité de notre outil d'analyse descriptif. Nos recommandations seront divisées en cinq thèmes principaux soient l'analyse des commandes clients, l'analyse des ventes par région, l'analyse des expéditions par région, l'analyse des régions desservies et finalement, l'analyse des clients perdus.

**3.1 Analyse des commandes clients**

Tout d'abord, de manière globale, il est possible de constater qu'un peu moins de 10 000 commandes ont été effectuées auprès de l'entreprise en quatre ans. La moyenne du montant de ces commandes est de 230,12 dollars alors que le plus petit montant pour une commande a été de 0,44 dollar et le plus grand montant a été de 22 638,48 dollars. Plus précisément, pour ce qui est des trois segments de consommateurs, il est possible de constater que les particuliers ont des commandes moyennes de 225,02 dollars avec un montant maximal de 13 999,96 dollars et un montant minimal de 0,44 dollar, les entreprises ont des commandes moyennes de 231,42 dollars avec un montant maximal de 17 499,95 dollars et un montant minimal de 0,56 dollar et les bureaux à la maison ont des commandes moyennes de 242,80 dollars avec un montant maximal de 22 638,48 dollars et un montant minimal de 0,99 dollar. 

Ensuite, pour les expéditions, il est possible de constater que 52% des expéditions de l'entreprise sont pour des particuliers, 30% pour des entreprises et 18% pour des bureaux à la maison. 

Pour les différents types de transports, il est possible de constater que la répartition des expéditions est pratiquement la même pour les trois segments de clients. En effet, peu importe le segment de client, le type de livraison standard est utilisé à approximativement 60%, le type deuxième classe à 20%, le type première classe à 15% et finalement le type journée même à 5%. 

Au niveau de la popularité des sous-catégories de produits en fonction du segment de client, il est possible de constater que la répartition est très semblable dans les trois cas. Effectivement, pour les trois segments de clients, les produits les plus populaires sont les cartables, le papier et les fournitures alors que les deux moins populaires sont les machines et les copieurs.

Sur le plan stratégique, les résultats obtenus en ce qui concerne les commandes clients nous permettent d'effectuer plusieurs recommandations. Premièrement, il est possible de constater qu'actuellement plus de la majorité des expéditions effectuées par l'entreprise sont pour des particuliers. Il est donc évident qu'il pourrait être intéressant pour les dirigeants de l'entreprise de favoriser leurs opérations en fonction de ce segment client, surtout en prenant en compte l'explosion du commerce électronique au cours des dernières années. De plus, il est possible de constater que le montant moyen des commandes est pratiquement le même pour les trois segments de clients, donc les revenus de l'entreprise ne seraient pas réellement affectés négativement si celle-ci décidait de se concentrer majoritairement sur le segment des particuliers. Deuxièmement, pour ce qui est des types de livraison préconisés par les différents segments de clients, il est possible de constater que leur comportement ne diffère pas d'un segment de client à l'autre contrairement à ce que nous aurions pu croire. Il est également possible de constater que les clients de l'entreprise ne semblent pas réellement voir d'avantage à opter pour la livraison de type journée même. Il pourrait donc être intéressant de réviser le prix de ce type de livraison à la baisse afin que les clients soient plus portés à le choisir. Surtout si les dirigeants optent afin de favoriser le commerce en ligne pour les particuliers, ce type de livraison offert à un prix adéquat pourrait devenir un très grand avantage compétitif. Finalement, notre troisième recommandation serait axée sur les sous-catégories de produits. Encore une fois, si l'entreprise décide de se concentrer davantage sur le commerce en ligne, il pourrait être pertinent de laisser tomber la vente de produits comme les copieurs et les machines. Effectivement, il s'agit de produits volumineux qui pourraient générer des coûts très élevés en transport et nos analyses ont démontré qu'il s'agissait de produits très peu populaires auprès des différents segments de clients. L'entreprise pourrait donc plutôt se concentrer sur des produits à plus faible volume tels que les cartables, le papier et les fournitures qui représentaient tous des produits très populaires auprès des trois segments de clients. 


**3.2 Analyse des ventes selon les territoires**

Tout d'abord, grâce à la carte interactive représentant les ventes totales en dollars pour les différents états, il est possible de constater que les cinq principaux états où l'entreprise a effectué les plus grands totaux en ventes sont la Californie avec 446k dollars,New York avec 306k dollars, le Texas avec 169k dollars, Washington avec 135k dollars et finalement la Pennsylvanie avec 116k dollars. 

Sur le plan stratégique, les résultats obtenus en ce qui concerne l'analyse des ventes en dollars selon les territoires nous permettent d'effectuer une recommandation principale aux dirigeants de l'entreprise. Bien évidemment, cette recommandation serait de favoriser un service client de la plus grande qualité possible auprès des clients étant situés dans les cinq états principaux soient la Californie, New York, le Texas, Washington et la Pennsylvanie. Il pourrait donc être intéressant pour les dirigeants de l'entreprise de favoriser leurs efforts marketing dans ces régions étant donné qu'il s'agit de leurs marchés principaux. Pour ce qui est de la décision concernant l'emplacement des centres de distribution de l'entreprise, il est évident que ceux-ci devront être situés à proximité des ces cinq états afin d'offrir un service rapide aux clients s'y trouvant, toutefois c'est plutôt l'aspect du nombre d'expéditions par état qui sera favorisé afin de prendre une décision par rapport à ce sujet étant donné que les ventes en dollars peuvent être influencées par des produits ayant un prix de vente élevé. En effet, d'un point de vue logistique, il est plus pertinent d'évaluer l'emplacement possible d'un centre de distribution en tentant de minimiser le nombre de déplacements et maximisant le nombre de clients auxquels l'entreprise a accès, ce qui permet de limiter les coûts en transport. 

**3.3 Analyse des expéditions selon les territoires**

Cette fois-ci, en observant plutôt le nombre de commandes expédiées vers les différents états, il est possible de constater que les cinq mêmes états composent toujours le top 5 des états principaux pour l'entreprise. L'ordre d'importance a toutefois changé. Le top trois reste inchangé avec la Californie en première place avec 1946 expéditions, New York en deuxième plage avec 1097 expéditions et le Texas en troisième place avec 973 expéditions. La Pennsylvanie est maintenant en quatrième position avec 582 expéditions et Washington occupe le cinquième rang avec 504 expéditions. En regardant plus attentivement la carte, il est possible de repérer que les expéditions de l'entreprise sont majoritairement concentrées au Nord-Est avec la région de l'état de New York, au Sud avec l'état du Texas et celui de la Floride et finalement à l'Ouest avec les états de la Californie et de Washington. 

Ensuite, la préférence des sous-catégories de produits a été évaluée en fonction des différentes régions desservies par l'entreprise soit le Centre, l'Est, l'Ouest et le Sud. De manière globale, il a été possible de constater que la répartition du pourcentage des expéditions entre les différentes sous-catégories de produits était plutôt semblable pour les quatre régions. En effet, les produits les plus populaires étaient toujours les classeurs, le papier, l'entreposage, les téléphones et les fournitures alors que les produits les moins populaires étaient les machines et les copieurs. 

Au niveau des villes les plus importantes en termes de nombre d'expéditions effectuées pour des clients se retrouvant dans ces villes, il est possible de constater que New York City occupe la première place avec 891 expéditions, suivi de Los Angeles avec 728 expéditions, de Philadelphie avec 532 expéditions, de San Francisco avec 500 expéditions et finalement de Seattle avec 426 expéditions. 

En s'attardant sur les trois états principalement desservis par l'entreprise soit la Californie, New York et le Texas, il est possible de ressortir trois villes principales pour chacun d'eux. Pour l'état de la Californie, les trois principales villes où l'entreprise effectue des expéditions sont Los Angeles avec 728, San Francisco avec 500 et San Diego avec 170. Pour l'état de New York, les trois principales villes sont New York City avec 891, Rochester avec 36 et Long Beach avec 32. Finalement, pour l'état du Texas, les trois principales villes sont Houston avec 374, Dallas avec 156 et San Antonio avec 59.

Sur le plan stratégique, les résultats obtenus en ce qui concerne les expéditions par territoire nous permettent d'effectuer plusieurs recommandations. Premièrement, il est possible de constater qu'il pourrait être intéressant pour l'entreprise de séparer ses expéditions selon trois centres de distribution distincts. Ce premier centre de distribution pourrait être situé dans la partie supérieure de l'état de la Californie afin de pouvoir couvrir les expéditions de la région ouest du pays. Ce centre de distribution pourrait donc être situé dans la ville de San Francisco qui représente une des cinq villes les plus importantes au niveau des expéditions. Le deuxième centre de distribution pourrait être situé dans la partie supérieure de l'état du Texas afin de pouvoir couvrir les expéditions de la région sud du pays. Ce centre de distribution pourrait donc être situé dans la ville de Dallas qui représente une des principales villes situées dans la partie supérieure de cet état. Malgré le fait qu'elle n'ait pas été sélectionnée parmi les cinq principales villes desservies par l'entreprise, cette ville semble être un emplacement stratégique afin de couvrir la région Sud du pays. Le troisième centre de distribution pourrait être situé à la frontière entre les états de New York et de la Pennsylvanie afin de pouvoir couvrir les expéditions de la région Nord-Est du pays. Ce centre de distribution pourrait donc être situé dans la ville de Scranton qui se trouve dans la partie supérieure à l'est de l'état de la Pennsylvanie. Bien que cette ville n'ait pas été sélectionnée parmi les cinq villes les plus populaires de l'entreprise, elle semble être un bon compromis entre les villes de New York City et Philadelphie, qui elles représentent deux de ces cinq villes. La deuxième recommandation que nous pourrions faire aux dirigeants de l'entreprise serait de ne pas opter pour des centres de distribution spécialisés. En effet, nos analyses ont permis de constater que la répartition des expéditions au niveau des sous-catégories de produits était pratiquement identique pour les quatre régions desservies par l'entreprise. Les trois centres de distribution pourraient donc être conçus et opérés tous de la même manière, ce qui faciliterait grandement les décisions et la gestion des dirigeants de l'entreprise.

**3.4 Analyse des territoires desservis**

Tout d'abord, en regardant de manière précise l'emplacement de chacun des clients ayant effectué une commande auprès de l'entreprise au cours des quatre années pour lesquelles nous avons des données, il est possible de constater que l'entreprise est en mesure de couvrir tout le territoire des États-Unis, bien que les commandes soient majoritairement concentrées dans les régions mentionnées dans la section précédente. Il est même possible de constater que l'entreprise a effectué quelques expéditions pour des clients au Canada, pour un client en Équateur et finalement un client en Angleterre. 

Sur le plan stratégique, les résultats obtenus en ce qui concerne les territoires desservis par l'entreprise nous permettent d'effectuer plusieurs recommandations. Pour notre première recommandation, il pourrait être intéressant que les dirigeants de l'entreprise évaluent s'il est réellement pertinent de couvrir tout le territoire des États-Unis comme c'est le cas actuellement. En effet, il est possible de constater qu'une grande portion de territoire se trouvant au centre du pays n'a pratiquement aucun client. Les commandes expédiées dans cette région doivent donc coûter énormément cher au niveau des coûts de transport. Si les dirigeants de l'entreprise veulent absolument couvrir tout le territoire américain, il pourrait donc être intéressant d'instaurer un coût minimal de commande pour ces régions où il y a très peu de clients afin de rentabiliser leurs expéditions. Autrement, ces régions ne devraient plus être desservies par l'entreprise d'un point de vue logistique étant donné que les profits tirés des ces clients doivent être minimes voir même inexistants. Notre deuxième recommandation serait axée au niveau des expéditions internationales effectuées par l'entreprise. Dans le même sens que les expéditions effectuées dans les régions où se trouvent peu de clients, l'entreprise devra évaluer de plus près l'option d'expédier des commandes à l'international. Comme l'entreprise possède très peu de clients à l'extérieur des États-Unis actuellement et que le transport international demande des qualifications supplémentaires, il serait préférable de concentrer les activités de l'entreprise à l'intérieur des États-Unis à moins d'augmenter de manière considérable leur base de clients internationaux et d'en faire une spécialisation.

**3.5 Analyse des clients perdus**

Tout d'abord, comme mentionné dans la méthodologie, cette analyse a été créée avec l'aide de paramètres modifiables afin que les dirigeants puissent évaluer plusieurs scénarios différents. Nous avons donc analysé le cas où les dirigeants voudraient évaluer la situation des clients n'ayant pas effectué une commande au cours de la dernière année. En observant la carte représentant les expéditions effectuées par le passé à ces clients n'ayant pas commandé auprès de l'entreprise dans la dernière année, il est possible de constater qu'ils sont distribués également un peu partout au travers des États-Unis. Toutefois, en portant attention à la "Heat map", il est possible de constater que ces clients ayant possiblement été insatisfaits de leur achat auprès de l'entreprise sont majoritairement situés au Nord-Est du pays. Par contre, historiquement plus de commandes sont effectuées dans cette région, ce qui pourrait également expliquer que la concentration des clients perdus de cette région soit plus élevée. Il ne semble donc pas y avoir d'explication précise étant reliée à l'emplacement pour les clients perdus étant donné que ceux-ci représentent la distribution normale des clients de l'entreprise.

Ensuite, avec l'aide de la liste des commandes effectuées par ces clients perdus au cours de la dernière année, il est possible de constater que 528 de ces expéditions avaient été réalisées par le type de transport standard, 183 avec le type de transport deuxième classe, 124 avec le type de transport première classe et 33 avec le type de transport journée même. 

Sur le plan stratégique, les résultats obtenus en ce qui concerne les clients perdus nous permettent d'effectuer plusieurs recommandations. Pour notre première recommandation, il est possible de constater qu'il ne semble pas y avoir de problèmes majeurs d'insatisfaction par rapport aux différentes régions desservies par l'entreprise. Les dirigeants devraient donc continuer la gestion des territoires de la manière qu'elle est effectuée actuellement. Par contre, notre deuxième recommandation serait au niveau des types de transport ayant été utilisés par les clients perdus. Nos analyses ont dégagé le fait que la grande majorité des clients perdus avaient opté pour le type de livraisons standard lors de leurs dernières expéditions. Il pourrait donc être pertinent d'évaluer si les attentes des clients envers ce type de livraison sont réalistes ou si ceux-ci s'attendaient à un meilleur service en choisissant ce type de livraison. Afin de remédier à cette situation et regagner la confiance de ces clients, l'entreprise pourrait diminuer le tarif du type de livraison standard ou offrir une promotion à ces clients en leur offrant la livraison de type deuxième classe au prix de la livraison de type standard lors de leur prochaine commande.


# 4. Conclusion : Limites, pistes d'amélioration et enjeux éthiques 

Afin de conclure ce rapport, trois thèmes principaux seront abordés dans le but d’évaluer l’outil d’analyse descriptif que nous avons créé dans le cadre de ce projet. Premièrement, les limites que nous avons pu constater lors de l’élaboration de nos analyses seront présentées. Ensuite, dans un deuxième temps nous effectuerons des suggestions par rapport à des pistes d’amélioration qui pourraient être intéressantes afin d’amener notre outil d’analyse à un niveau supérieur. Finalement, le dernier thème de cette section abordera les enjeux éthiques qui pourraient être mis en cause avec l’utilisation de notre outil d’analyse. 

**Limites :** 

Tout d’abord, la première limite que nous avons rencontrée lors de l’élaboration de cet outil d’analyse a été au niveau de l’accessibilité des informations que nous avions par rapport à l’entreprise. En effet, comme il s’agissait d’une base de données ayant été rendue publique de manière anonyme, nous ne pouvions avoir accès à des informations sur l’entreprise. Il est évident que si nous avions eu accès à de telles informations, nous aurions pu pousser l’analyse de nos résultats à un autre niveau en connaissant la culture et la réputation de l’entreprise. Aussi, le fait d’avoir accès à l’entreprise aurait pu nous permettre de valider certaines informations supplémentaires par rapport aux informations de la base de données.
	
Ensuite, la deuxième limite que nous avons rencontrée a été par rapport au manque de données concernant certaines commandes. Effectivement, lorsque nous avons voulu représenter l’emplacement d’expédition de certaines commandes, nous avons constaté que certaines d’entre elles n’avaient pas de code postal. Comme nous n’avions pas l’information par rapport à ces commandes, nous avons dû les éliminer de la base de données. De ce fait, il est donc possible de que nos analyses pour certaines régions aient été faussées étant donné que des commandes ont été enlevées de la base de données. 

Finalement, la dernière limite que nous avons rencontrée a été le fait que notre base de données datait des années 2015 à 2018 alors que notre outil était utilisé afin d’effectuer des recommandations sur le plan stratégique pour l’entreprise. Il est évident qu’afin d’effectuer de telles recommandations, il serait primordial d’avoir accès à des données le plus récentes possible. Toutefois, lorsque l’entreprise nous donnerait accès à ces données plus récentes, il serait facile de simplement mettre à jour la base de données utilisée par notre outil d’analyse afin de générer de nouveaux résultats plus représentatifs de la situation actuelle de l’entreprise.


**Pistes d’amélioration :** 

Tout d’abord, la première piste d’amélioration que nous pouvons suggérer serait de procéder à l’élaboration d’un tableau de bord permettant de visualiser rapidement et de manière concise les différents graphiques et analyses que nous avons réalisés lors de la création de cet outil. Actuellement, nous sommes conscients qu’il peut être fastidieux de devoir naviguer dans l’outil afin d’avoir accès à l’information désirée. En ayant accès à un tableau de bord, les utilisateurs pourraient rapidement avoir accès aux diverses informations nécessaires afin d’effectuer des recommandations stratégiques lors des réunions de direction. 

Ensuite, une deuxième piste d’amélioration qui pourrait être intéressante à élaborer serait d’identifier les saisonnalités au niveau de la demande. Effectivement, nos analyses n’ont pas couvert cet aspect, mais il pourrait être pertinent dans le but de développer nos recommandations stratégiques d’évaluer les périodes pour lesquelles la demande de certains produits est grandement supérieure à la normale. En ayant accès à cette analyse, il serait plus facile pour l’entreprise de prévoir les niveaux de stocks nécessaires afin de subvenir à la demande des clients et du même coup il serait plus facile de prévoir la capacité nécessaire pour les centres de distribution. Cette analyse pourrait également être utile afin de déterminer les besoins en main-d’œuvre de chacun des centres de distribution en fonction des niveaux de demande des différentes périodes de l’année. 

Finalement, la dernière piste d’amélioration que nous avons soulevée serait d’améliorer la rapidité d’exécution de notre outil d’analyse. Comme nous utilisons une version gratuite du module nous permettant de représenter l’emplacement de nos clients sur une carte, un long lapse de temps est nécessaire afin de générer la carte. Comme notre outil est utilisé afin d’effectuer des recommandations stratégiques pour lesquelles une réponse rapide n’est pas cruciale, il ne s’agit pas d’un enjeu majeur pour notre outil. Toutefois, il serait tout de même intéressant que l’entreprise opte pour la version payante afin d’optimiser la rapidité de l’outil.

**Enjeux éthiques :**
À travers ce travail et en nous basant sur la déclaration de Montréal pour un développement responsable de l'intelligence artificielle nous avons identifié trois enjeux éthiques.
Le premier principe dont nous devons tenir compte est celui de protection de l’intimité et de la vie privée. La déclaration de Montréal avance deux sous enjeux à ce principe qu’il nous semble intéressant de discuter. Les deux principes sont les suivants : (1) « Les SAAD doivent garantir la confidentialité des données et l’anonymisation des profils personnels. » (2) « Toute personne doit pouvoir garder un contrôle étendu sur ses données personnelles, en particulier par rapport à leur collecte, usage et dissémination. L’utilisation par des particuliers de SIA et de services numériques ne peut être conditionnée à l’abandon de la propriété de ses données personnelles. »
À partir du fichier, nous disposons de beaucoup d’information sur nos clients notamment leur nom et leur adresse qui peuvent être des données privées. Il est donc important dans le but de conserver l’anonymat de nos clients de ne pas les identifier avec leur nom, mais de privilégier l’utilisation de leur numéro client. De plus, il sera important de contrôler les accès à la base de données et ajoutant des codes d’accès distribuer uniquement aux personnes en ayant besoin. On pourrait avoir plusieurs des bases de données distinctes pour ne pas conserver le nom des clients et leur adresse dans la même base de données.  

Le second principe identifié est celui de respect de l’autonomie. Deux sous-enjeux de cette catégorie nous semblent intéressants à intégrer dans notre discussion. Il s’agit de (1) « Les SIA doivent permettre aux individus de réaliser leurs propres objectifs moraux et leur conception de la vie digne d’être vécue. » et (2) « Les SIA ne doivent pas être développés ni utilisés pour prescrire aux individus un mode de vie particulier, soit directement, soit indirectement en mettant en œuvre des mécanismes de surveillance, d’évaluation ou d’incitation contraignants. ». En effet, nous avons utilisé le fichier pour être en mesure d’identifier des tendances de comportement chez nos clients et cibler les produits les plus demandés pour chaque type de client afin d’avoir un marketing assez ciblé. Il faudra donc veiller à toujours laisser le choix à nos clients de recevoir ou non nos campagnes marketing ciblées et de leur laisser le libre choix de participer ou non à ces campagnes.
De plus, l’analyse réalisée sur la stratégie de réacquisition de nos clients perdus peut s’avérer problématique. En effet, on essaye de comprendre pourquoi nos clients nous ont quittés dans le but de leur faire des offres ciblées et de les inciter à consommer ce qui peut nuire à leur autonomie et leur libre-choix.

Le dernier enjeu éthique qu’il nous semble pertinent d’intégrer de discuter est le principe de prudence.  « Le développement des SIA doit prévenir les risques d’une utilisation néfaste des données d’utilisateurs et protéger l’intégrité et la confidentialité des données personnelles. » (Déclaration de Montréal pour un développement responsable de l'intelligence artificielle,2018). Nous offrons avec ce travail une description détaillée du comportement de leurs clients à la compagnie. Comme mentionné dans le précédent enjeu, avec ces données, on peut exposer les clients à un certain harcèlement commercial de la compagnie. Afin d'éviter cette situation, nous pourrions partager à la compagnie uniquement les analyses de la consommation par ville et par État, qui sont déjà assez révélatrices de certaines tendances de consommation. Ainsi, la compagnie bénéficierait d’information juste et intéressante pour prendre des décisions stratégiques sur son réseau de distribution tout en garantissant l’anonymat des clients. 

# 5. Contribution des membres de l'équipe

Dans le cadre de ce travail, nous avons veillé à une répartition équitable des tâches. Dès les premières rencontres, lors du choix du sujet et de la définition de l’objectif du projet, chacun a partagé son expérience en Python et son niveau d’aisance avec le code. Nous souhaitions que chacun participe au travail, mais que les analyses demandées soient réalistes quant aux habilités de code de chacun. 

Dans notre équipe, deux membres étaient plus à l’aise avec le code que deux autres. Ainsi, nous avons confié le soin à ces deux coéquipiers de s’occuper du nettoyage de la base de données et de s’assurer que le fichier que nous allions utiliser pour poursuivre les analyses était « propre ». Afin d’équilibrer la charge de travail, nous avons convenu que les deux autres membres de l’équipe s’occuperaient de rédiger une plus grande partie du rapport. Une fois la base de données nettoyée et utilisable, nous avons convenu des analyses que nous souhaitions faire.  Nous avons tous les quatre réalisé quelques analyses communes pour nous assurer que nous avions les bons résultats et qu’il n’y avait pas d’incohérence dans les données.  Nous nous sommes ensuite réparti une ou deux sous-parties de l’analyse pour être en mesure d’identifier des tendances dans les commandes selon le type de client, les territoires desservis et les catégories de produits. 

Afin de ne pas nous gêner ou de casser les codes des uns et des autres, nous travaillions chacun sur des notebooks Kaggle distincts, mais nous nous étions partagé les accès afin de suivre les avancées de chacun et établir des liens entre nos analyses. Depuis la mi-session, nous nous rencontrions également chaque semaine pour mettre en commun notre travail, valider les analyses réalisées et nous partager le travail restant. Ces réunions ont été une force dans notre travail car elles permettaient que chacun présente ses lignes de code et en cas d’erreur ou de problèmes, nous pouvions nous entraider. Ceux plus à l’aise en code pouvaient aussi aider ceux moins à l’aise. De plus, comme nous faisions chacun nos analyses de notre côté, lors de la mise en commun nous pouvions nous partager des techniques et des astuces apprises en code ! 

Finalement pour la rédaction du rapport final, nous avons veillé à laisser des commentaires sur le code ou sous nos lignes de codes pour que tout le monde soit en mesure de comprendre le code et d’expliquer chaque partie du rapport. Chacun a ainsi rédigé une section du rapport les deux personnes n’ayant pas participé au nettoyage des données ayant pris les plus grosses parties du rapport.